<a href="https://colab.research.google.com/github/alan-medina-gomez-lic/Evidencia1_Ago2025/blob/main/RDBMS_ManejoFechas_SQLite.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Manejo de fechas en SQLite

##Creación de tablas con columnas para fechas

Es importante considerar que, si deseamos manejar fechas en nuestros datos, las columnas destinadas para tal propósito deberán crearse como tipo `timestamp`. Observe el siguiente código, especialmente para la definición de la columna `fecha_registro`.

In [ ]:
import datetime
import sqlite3
import sys

try:
    with sqlite3.connect("ConFechas_timestamp.db") as conexion:
        mi_cursor = conexion.cursor()
        mi_cursor.execute("CREATE TABLE amigo (clave INTEGER PRIMARY KEY, \
        nombre TEXT NOT NULL, \
        fecha_registro timestamp);")
        print("Tabla creada")
except sqlite3.Error as e:
    print(e)
except:
    print(f"Se produjo el siguiente error: {sys.exc_info()[0]}")
finally:
    if (conexion):
        conexion.close()
        print("Se ha cerrado la conexión")

##Agregado de registros con valores de fecha

Los valores de fecha con el tipo `datetime` de Python simplemente se incluyen como tales en los datos a almacenar. Observe la línea en la que se integra la tupla que será la fuente de valores para el `INSERT` correspondiente.

Considere que, en caso de solicitar los datos al usuario como usualmente ocurrirá, se deberá convertir la entrada tipo `str` al tipo de dato especializado para el manejo de fechas en Python pudiendo especificar el formato en que se desee manejar la fecha en sí.

In [ ]:
import datetime
import sqlite3
import sys

try:
    with sqlite3.connect("ConFechas_timestamp.db") as conexion:
        mi_cursor = conexion.cursor()
        datos = (20, "Pepo", datetime.datetime(2025,10,21))
        mi_cursor.execute("INSERT INTO amigo VALUES (?,?,?)", datos)
        print("Registro agregado exitosamente")

except sqlite3.Error as e:
    print(e)
except:
    print(f"Se produjo el siguiente error: {sys.exc_info()[0]}")

##Lectura de datos de tipo fecha

Para que SQLite interprete correctamente que manejará datos que contienen fechas se recomienda aprovechar el parámetro `detect_types` aplicable al método de apertura de la conexión. En el referido parámetro se le debe asignar el valor de las siguientes constantes del módulo sqlite3 `sqlite3.PARSE_DECLTYPES | sqlite3.PARSE_COLNAMES`

Al configurar la conexión de esa manera, SQLite retornará los valores que contiene fechas como el tipo especializado correspondiente de Python. Tenga presente también que es muy importante que para que esta funcionalidad tenga el resultado esperado, las columnas pertinentes debieron haber sido declaradas de tipo `timestamp`

Una vez que ya se tenga el valor recuperado, se puede formatear según se requiera como cualquier otro valor `datetime` de Python

In [ ]:
from datetime import date, datetime
import sqlite3
from sqlite3 import Error
import sys

#Es MUY importante que las columnas para fecha, se establezcan como timestamp en SQLite
#Esto para asegurar que se puedan convertir directamente a variables datetime de Python

try:
    with sqlite3.connect("ConFechas_timestamp.db",
                        detect_types = sqlite3.PARSE_DECLTYPES | sqlite3.PARSE_COLNAMES) as conn:
    #with sqlite3.connect("ConFechas_timestamp.db") as conn:
        mi_cursor = conn.cursor()
        mi_cursor.execute("SELECT clave, nombre, fecha_registro FROM Amigo")
        registros = mi_cursor.fetchall()

        for clave, nombre, fecha_registro in registros:
            print(f"Clave = {clave}, tipo de dato {type(clave)}")
            print(f"Nombre = {nombre}, tipo de dato {type(nombre)}")
            print(f"Fecha de registro = {fecha_registro.date().strftime('%d/%m/%Y')},\
             tipo de dato {type(fecha_registro)}\n")
            #print(f"Fecha de registro = {fecha_registro}, tipo de dato {type(fecha_registro)}\n")

except sqlite3.Error as e:
    print (e)
except Exception:
    print(f"Se produjo el siguiente error: {sys.exc_info()[0]}")

##Consulta utilizando la fecha en el criterio de discriminación

Tenga presente, para motivos de comparación, que al momento de almacenar la fecha en una columna datetime esta incluirá también la parte de la hora y esto puede ocasionar resultados de comparación diferentes a la intención original. Tal situación se puede manejar facilmente mediante el uso de la función `DATE()` de SQL.

In [ ]:
import datetime
import sqlite3
from sqlite3 import Error
import sys
#Es MUY importante que las columnas para fecha, se establezcan como timestamp en SQLite
#Esto para asegurar que se puedan convertir directamente a variables datetime de Python

fecha_consultar = input("Dime una fecha (dd/mm/aaaa): ")
fecha_consultar = datetime.datetime.strptime(fecha_consultar, "%d/%m/%Y").date()

try:
    with sqlite3.connect("ConFechas_timestamp.db",
                         detect_types = sqlite3.PARSE_DECLTYPES | sqlite3.PARSE_COLNAMES) as conn:
        mi_cursor = conn.cursor()
        criterios = {"fecha":fecha_consultar}
        mi_cursor.execute("SELECT clave, nombre, fecha_registro FROM Amigo \
        WHERE DATE(fecha_registro) = :fecha;", criterios)
        #mi_cursor.execute("SELECT clave, nombre, fecha_registro FROM Amigo \
        #WHERE DATE(fecha_registro) >= :fecha;", criterios)
        registros = mi_cursor.fetchall()

        if registros:
            for clave, nombre, fecha_registro in registros:
                print(f"Clave = {clave}, tipo de dato {type(clave)}")
                print(f"Nombre = {nombre}, tipo de dato {type(nombre)}")
                print(f"Fecha de registro = {fecha_registro.date()}, tipo de dato {type(fecha_registro)}\n")
        else:
            print("\nNO SE ENCONTRARON DATOS PARA ESA FECHA")

except sqlite3.Error as e:
    print (e)
except Exception:
    print(f"Se produjo el siguiente error: {sys.exc_info()[0]}")
finally:
    if (conn):
        conn.close()
        print("Se ha cerrado la conexión")